Блок 1: Импортиране на библиотеки

In [1]:
# Импортираме необходимите библиотеки
import json
import sqlite3
import pandas as pd
from IPython.display import clear_output
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
import numpy as np

Блок 2: Зареждане на данните

In [2]:
# Зареждаме JSON данните и ги преобразуваме в pandas DataFrame
def load_books(file_path):
    """
    Зарежда данни за книги от JSON файл и ги преобразува в pandas DataFrame.
    """
    with open(file_path, "r", encoding="utf-8") as file:
        data = json.load(file)
    books = pd.DataFrame.from_dict(data, orient="index")
    books.reset_index(drop=True, inplace=True)
    return books

Блок 3: Изчисляване на сходства между книгите

In [4]:
# Изчисляваме матрица на сходства между книгите
def compute_similarity(books, use_word2vec=False):
    """
    Изчислява матрица на сходствата между книгите.
    Използва TF-IDF или Word2Vec в зависимост от параметъра use_word2vec.
    """
    # Вземаме текстовете на книгите от полето "Text" в JSON данните
    book_texts = books["Text"].tolist()

    if use_word2vec:
        # Създаваме и обучаваме Word2Vec модел
        sentences = [text.split() for text in book_texts]  # разделяме на думи
        model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

        # Преобразуваме текста на всяка книга в средна стойност на векторите на думите
        book_vectors = []
        for text in sentences:
            vectors = [model.wv[word] for word in text if word in model.wv]
            if vectors:
                book_vectors.append(np.mean(vectors, axis=0))  # среден вектор
            else:
                book_vectors.append(np.zeros(100))  # ако няма думи в модела, връща нулев вектор

        # Изчисляваме косинусовата близост между книгите
        similarity_matrix = cosine_similarity(book_vectors)
    
    else:
        # Използваме TF-IDF векторизация
        vectorizer = TfidfVectorizer(stop_words='english')
        tfidf_matrix = vectorizer.fit_transform(book_texts)
        
        # Изчисляваме косинусовата близост между книгите
        similarity_matrix = cosine_similarity(tfidf_matrix)

    np.fill_diagonal(similarity_matrix, 1.0)  # Задаваме сходството между книгата и самата нея на 1.0
    return similarity_matrix

Блок 4: Функция за препоръки

In [5]:
# Функция, която предлага препоръчани книги, базирани на търсено заглавие
def get_recommendations(book_title, books, similarity_matrix, top_n=5):
    """
    Намира топ N книги, най-близки до подадената книга.
    """
    if book_title not in books['Title'].values:
        print(f"Книгата '{book_title}' не е намерена в базата.")
        return pd.DataFrame()
    
    book_idx = books[books['Title'] == book_title].index[0]
    similarity_scores = list(enumerate(similarity_matrix[book_idx]))
    
    # Сортиране по сходство (според втората стойност на всяка кортеж)
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    
    # Вземаме топ N препоръки, но не трябва да излизаме извън дължината на списъка
    top_n = min(top_n, len(similarity_scores) - 1)  # -1, за да не включваме самата книга
    similarity_scores = similarity_scores[1:top_n+1]  # Премахваме самата книга (индекс 0)

    # Получаваме индексите на препоръчаните книги
    recommended_indices = [idx for idx, _ in similarity_scores]

    # Проверяваме дали индексите са валидни и се уверяваме, че не излизат извън рамките
    if any(idx >= len(books) for idx in recommended_indices):
        print("Грешка: Някои от препоръчаните индекси не са валидни.")
        return pd.DataFrame()
    
    return books.iloc[recommended_indices]

Блок 5: Оценка на препоръките

In [6]:
# Функция за изчисляване
def evaluate_recommendations(books, similarity_matrix, book_idx, k=5):
    """
    Изчислява метриките P@k, R@k и MAP, използвайки релевантни книги.
    """
    # Извличаме релевантните книги (със съвпадение на жанровете)
    relevant_books = set(books[books['Genres'].str.contains(books.iloc[book_idx]['Genres'], case=False, na=False)].index)
    
    # Извличаме топ k препоръки от матрицата на сходствата
    retrieved_books = set(np.argsort(similarity_matrix[book_idx])[-k:])
    
    # Пресмятаме Precision и Recall
    intersection = relevant_books & retrieved_books
    precision = len(intersection) / k
    recall = len(intersection) / len(relevant_books) if relevant_books else 0
    
    # Изчисляваме Average Precision (MAP)
    avg_precision = sum([(i+1) / (idx+1) for i, idx in enumerate(intersection)]) / k if relevant_books else 0
    
    return precision, recall, avg_precision

Блок 6: Записване на резултатите в SQLite

In [7]:
# Функция за записване на резултатите в база данни
def save_results_to_sqlite(search_title, recommendations, books, db_path="../database/books_recommendations.db"):
    """
    Записва резултатите от препоръките и информацията за книгите в SQLite база данни.
    """
    SAVE_ALL_BOOKS = False  # True, ако решим да записваме всички книги в базата
    try:
        with sqlite3.connect(db_path) as conn:
            cursor = conn.cursor()

            # Създаване на таблица за препоръки
            cursor.execute(""" 
                CREATE TABLE IF NOT EXISTS recommendations (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    searched_title TEXT,
                    recommended_titles TEXT,
                    search_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                )
            """)

            # Записване на резултатите от препоръките
            cursor.execute(""" 
                INSERT INTO recommendations (searched_title, recommended_titles)
                VALUES (?, ?)
            """, (search_title, ", ".join(recommendations)))

            if SAVE_ALL_BOOKS:
                # Създаване на таблица за книги
                cursor.execute(""" 
                    CREATE TABLE IF NOT EXISTS books (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        title TEXT,
                        author TEXT,
                        genres TEXT
                    )
                """)
                
                # Извличаме само заглавията, авторите и жанровете от DataFrame
                books_data = [(book["Title"], book["Author"], book["Genres"]) for _, book in books.iterrows()]
                
                # Вмъкваме книгите в базата, като пропускаме вече съществуващите записи
                cursor.executemany(""" 
                    INSERT OR IGNORE INTO books (title, author, genres)
                    VALUES (?, ?, ?)
                """, books_data)

            conn.commit()  # Потвърждаваме промените в базата данни
        print("\nРезултатите са записани в базата данни.")
    except sqlite3.Error as e:
        print(f"\nГрешка при запис в базата данни: {e}")

Блок 7: Основна функция и изпълнение

In [9]:
def main():
    """
    Основна функция, която зарежда книги, изчислява сходства и предоставя препоръки.
    """
    clear_output(wait=True)
    
    file_path = "../archive/data.json"
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            pass  # Ако файлът може да бъде отворен, продължаваме
    except FileNotFoundError:
        print(f"Файлът '{file_path}' не беше намерен.")
        return

    books = load_books(file_path)
    similarity_matrix = compute_similarity(books, use_word2vec=True)
    
    user_input = input("Въведете заглавие на прочетена книга: ").strip()
    
    if not user_input:
        print("Моля, въведете заглавие на книга.")
        return
    
    if user_input not in books['Title'].values:
        print(f"Книгата '{user_input}' не беше намерена в базата.")
        available_books = books['Title'].head(10).tolist()  # Показване на първите 10 книги
        print(f"Може би имате предвид тези книги: {', '.join(available_books)}")
        return
    
    print("Търсене на препоръки...")
    recommendations = get_recommendations(user_input, books, similarity_matrix)

    if not recommendations.empty:
        print("Препоръчани книги:")
        display(recommendations)
    else:
        print("Няма намерени препоръки.")
    
    # Изчисляваме и показваме метриките
    p_at_k, r_at_k, map_score = evaluate_recommendations(books, similarity_matrix, books[books['Title'] == user_input].index[0])
    print(f"P@k: {p_at_k:.2f}, R@k: {r_at_k:.2f}, MAP: {map_score:.2f}")

    # Записване на резултатите в база данни
    save_results_to_sqlite(user_input, recommendations["Title"].tolist(), books)

# Изпълнение на основната функция
main()


Търсене на препоръки...
Препоръчани книги:


,Id,Title,Author,Year,Form,Genres,Text
1,2,Последната битка,Дейвид Едингс,1984,Роман,Епическо фентъзи Фентъзи,Епическо фентъзи Фентъзи Роман
20,21,Пазителите на запада,Дейвид Едингс,1987,Роман,Епическо фентъзи Фентъзи,Епическо фентъзи Фентъзи Роман
106,107,Пророчеството,Дейвид Едингс,1982,Роман,Епическо фентъзи Фентъзи,Епическо фентъзи Фентъзи Роман
150,151,Кралят на мургите,Дейвид Едингс,1988,Роман,Епическо фентъзи Фентъзи,Епическо фентъзи Фентъзи Роман
247,248,Господарят демон на Каранда,Дейвид Едингс,1988,Роман,Епическо фентъзи Фентъзи,Епическо фентъзи Фентъзи Роман


P@k: 1.00, R@k: 0.06, MAP: 0.20

Резултатите са записани в базата данни.
